In [24]:
import pandas as pd
import numpy as np

In [25]:
df_name_list = ["BET", "BUX", "EGX30", "NGXINDX", "Warsaw Stock Exchange WIG Index"]

In [26]:
for name in df_name_list:
    df_daily = pd.read_excel(name + " daily prices.xlsx")
    df_daily["Date"] = df_daily["Date"].astype("datetime64[D]")

    last_trade_date = []
    O_Date = df_daily.loc[0, "Date"]
    Price = df_daily.loc[0, "Last Price"]
    last_trade_date.append((O_Date.date(), Price))
    for i in range(len(df_daily)):
        Date = df_daily.loc[i, "Date"]
        Price = df_daily.loc[i, "Last Price"]
        if Date.month != O_Date.month or Date.year != O_Date.year:
            O_Date = Date
            last_trade_date.append((O_Date.date(), Price))
    
    temp1, temp2 = zip(*last_trade_date)
    df_monthly = pd.DataFrame({"Date": temp1, "Last_Price":temp2})

    df_mom_change = df_monthly.copy()
    for i in range(len(df_mom_change)-1):
        df_mom_change.loc[i, "mom%change"] = (df_mom_change.loc[i, "Last_Price"] - df_mom_change.loc[i+1, "Last_Price"]) / df_mom_change.loc[i+1, "Last_Price"] * 100
        
    df_mo24mma_change = df_monthly.copy()
    df_mo24mma_change = df_mo24mma_change.sort_values("Date")
    df_mo24mma_change["24mma"] = df_mo24mma_change["Last_Price"].rolling(24).mean()
    df_mo24mma_change = df_mo24mma_change.sort_values("Date", ascending=False)
    for i in range(len(df_mo24mma_change)-24):
        df_mo24mma_change.loc[i, "mo24mma%change"] = (df_mo24mma_change.loc[i, "Last_Price"] - df_mo24mma_change.loc[i+1, "24mma"]) / df_mo24mma_change.loc[i+1, "24mma"] * 100

    df_Volatility = df_daily[["Date", "Last Price"]]
    df_Volatility = df_Volatility.sort_values("Date")
    df_Volatility["20 days average"] = df_Volatility["Last Price"].rolling(20).mean()
    df_Volatility["Square of Difference"] = np.square(df_Volatility["Last Price"] - df_Volatility["20 days average"])
    df_Volatility["Square of Difference 20 days average"] = df_Volatility["Square of Difference"].rolling(20).mean()
    df_Volatility["Volatility"] = np.sqrt(df_Volatility["Square of Difference 20 days average"])
    df_Volatility = df_Volatility.sort_values("Date", ascending = False)
    
    df_mom_change.to_csv(name + "_mom%change.csv")
    df_mo24mma_change.to_csv(name + "_mo24mma_change.csv")
    df_Volatility.to_csv(name + "_Volatility.csv")


In [27]:
df_Volatility = df_daily[["Date", "Last Price"]]
df_Volatility = df_Volatility.sort_values("Date")
df_Volatility["20 days average"] = df_Volatility["Last Price"].rolling(20).mean()
df_Volatility["Square of Difference"] = np.square(df_Volatility["Last Price"] - df_Volatility["20 days average"])
df_Volatility["Square of Difference 20 days average"] = df_Volatility["Square of Difference"].rolling(20).mean()
df_Volatility["Volatility"] = np.sqrt(df_Volatility["Square of Difference 20 days average"])
df_Volatility = df_Volatility.sort_values("Date", ascending = False)
df_Volatility


,Date,Last Price,20 days average,Square of Difference,Square of Difference 20 days average,Volatility
0,2023-10-09,65008.48,65595.8750,3.450329e+05,2.034745e+06,1426.444848
1,2023-10-06,63903.35,65668.7535,3.116650e+06,2.147381e+06,1465.394558
2,2023-10-05,63799.77,65800.7030,4.003733e+06,2.142468e+06,1463.717151
3,2023-10-04,64282.18,65907.3605,2.641212e+06,2.271124e+06,1507.024973
4,2023-10-03,63776.83,66030.6070,5.079511e+06,2.329828e+06,1526.377457
...,...,...,...,...,...,...
5715,2000-12-13,17237.24,NaN,NaN,NaN,NaN
5716,2000-12-12,17088.13,NaN,NaN,NaN,NaN
5717,2000-12-11,16941.79,NaN,NaN,NaN,NaN
5718,2000-12-08,16860.59,NaN,NaN,NaN,NaN
